In [1]:
import pandas as pd
import numpy as np
import random
from rdkit import Chem
from snn_model import get_loss_fn
import torch
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from utils import load_dataset_df, smile_to_fp,smiles_to_descriptor,smiles_to_onehot, smiles_to_onehot_selfies, data_splitter, get_spiking_net, make_filename
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from csnn_model import get_prediction_fn
from torch.utils.data import random_split, TensorDataset
import torch.nn as nn
import deepchem as dc
from deepchem.splits.splitters import ScaffoldSplitter
import numpy as np

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\snn_venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometri

In [2]:
#!pip install git+https://github.com/scikit-fingerprints/scikit-fingerprints.git


In [3]:
#!pip freeze
#!pip install networkx==3.4.2

#### Load DataFrame

In [2]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[1]
dirname = dt_file.removesuffix('.csv')

df, targets = load_dataset_df(filename=dt_file)

for t in targets:
    df_temp = df[[t, 'smiles']].dropna()
    class_counts = df[t].count()
    class_sum = df[t].sum()
    print(t, class_counts, round(class_sum/class_counts, 2)) 


Hepatobiliary disorders 1427 0.52
Metabolism and nutrition disorders 1427 0.7
Product issues 1427 0.02
Eye disorders 1427 0.61
Investigations 1427 0.81
Musculoskeletal and connective tissue disorders 1427 0.7
Gastrointestinal disorders 1427 0.91
Social circumstances 1427 0.18
Immune system disorders 1427 0.72
Reproductive system and breast disorders 1427 0.51
Neoplasms benign, malignant and unspecified (incl cysts and polyps) 1427 0.26
General disorders and administration site conditions 1427 0.91
Endocrine disorders 1427 0.23
Surgical and medical procedures 1427 0.15
Vascular disorders 1427 0.78
Blood and lymphatic system disorders 1427 0.62
Skin and subcutaneous tissue disorders 1427 0.92
Congenital, familial and genetic disorders 1427 0.18
Infections and infestations 1427 0.7
Respiratory, thoracic and mediastinal disorders 1427 0.74
Psychiatric disorders 1427 0.71
Renal and urinary disorders 1427 0.64
Pregnancy, puerperium and perinatal conditions 1427 0.09
Ear and labyrinth disorde

In [3]:
if dirname == 'tox21':
    # SR-ARE
    target_name = targets[7]
    # SR-MMP
elif dirname == 'sider':
    #Hepatobiliary disorders 1427 samples, 0.52 class ratio
    target_name = targets[0]
else:
    target_name = targets[0]
    
df = df[[target_name, 'smiles']].dropna()

#### Molecular Representation

In [4]:
representations = ["fp", "descriptor", "SELFIES-1hot", "SMILES-1hot"]#, "graph-list"]

repr_type = representations[1]

In [ ]:
if repr_type == "fp":
    fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024], ['count_morgan', 1024], ['pubchem', 881]]
    mix = False
    fp_type, num_bits = fp_types[1]
    if mix and fp_type == 'RDKit':
        num_bits = 512
    data_config = {"fp_type": fp_type,
                "num_bits": num_bits,
                "radius": 2,
                "fp_type_2": fp_types[0][0],
                "num_bits_2": 1024 - num_bits,
                "mix": mix,}
    dim_2 = False
    print(fp_type, '-', num_bits)
    if mix: print(data_config['fp_type_2'], '-', data_config['num_bits_2'])
    if dim_2: print("2D FP")

elif repr_type == "descriptor":
    desc_type = ["RDKit", "Mordred"]
    data_config = {"desc_type": desc_type[1],
                   "size": 0,
                }
elif repr_type == "SELFIES-1hot":
    dim_2 = True
    data_config = {}

elif repr_type == "SMILES-1hot":
    dim_2 = True
    data_config = {}

data_config["repr_type"] = repr_type
print(repr_type)

descriptor


In [6]:
from utils import smiles_to_feat
dtype = torch.float
split = "scaffold" if dirname == "BBBP" else "random"
dataset = None


feat_tensor, target_tensor, feat_df = smiles_to_feat(df,repr_type=repr_type, data_config=data_config, target_name=target_name, dtype=dtype)

dataset = TensorDataset(feat_tensor, target_tensor)

[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:54] WARNING: not removing hydrogen atom without neighbors
[23:59:55] WARNING: not removing hydrogen atom without neighbors
[23:59:55] WARNING: not removing hydrogen atom without neighbors


IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
seed = 1
smiles_list = df["smiles"].tolist()
labels = df[target_name].values

Xs = np.zeros(len(smiles_list))
weights = np.zeros(len(smiles_list))

# Create a DiskDataset
dataset = dc.data.DiskDataset.from_numpy(X=Xs, y=labels, w=weights, ids=smiles_list)

scaffold_splitter = ScaffoldSplitter()
#by default is 0.8 0.1 0.1
#print("scaffold splitting..")
train, val,  test = scaffold_splitter.train_valid_test_split(dataset, seed=seed)
print(train)

[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors
[20:00:56] WARNING: not removing hydrogen atom without neighbors


<DiskDataset X.shape: (1141,), y.shape: (1141,), w.shape: (1141,), task_names: [0]>


In [10]:
print(train.ids)

['C(CNCCNCCNCCN)N' 'Cl[Tl]' 'C[N+](C)(C)CC(CC(=O)O)O' ...
 'C1CN(CCN1CCC2=C(C=C3C(=C2)CC(=O)N3)Cl)C4=NSC5=CC=CC=C54.O.Cl'
 'C1=CC(=CC=C1CCC2=CNC3=C2C(=O)N=C(N3)N)C(=O)N[C@H](CCC(=O)O)C(=O)O'
 'C[N+]1(CCC(C1)OC(=O)C(C2CCCC2)(C3=CC=CC=C3)O)C']


In [ ]:

train_ids_df = pd.DataFrame({'smiles': train.ids, target_name: train.y})
val_ids_df = pd.DataFrame({'smiles': val.ids, target_name: val.y})
test_ids_df = pd.DataFrame({'smiles': test.ids, target_name: test.y})
#print(len(train_ids_df), len(val_ids_df), len(test_ids_df))
#print("featurizing..")
fp_train, target_train = smile_to_fp(df=train_ids_df, data_config=data_config, target_name=target_name)
fp_val, target_val = smile_to_fp(df=val_ids_df, data_config=data_config, target_name=target_name)
fp_test, target_test = smile_to_fp(df=test_ids_df, data_config=data_config, target_name=target_name)

fp_train_tensor = torch.tensor(fp_train, dtype=dtype)
target_train_tensor = torch.tensor(target_train, dtype=dtype).long()
fp_val_tensor = torch.tensor(fp_val, dtype=dtype)
target_val_tensor = torch.tensor(target_val, dtype=dtype).long()
fp_test_tensor = torch.tensor(fp_test, dtype=dtype)
target_test_tensor = torch.tensor(target_test, dtype=dtype).long()


train = TensorDataset(fp_train_tensor, target_train_tensor)
val = TensorDataset(fp_val_tensor, target_val_tensor)
test = TensorDataset(fp_test_tensor, target_test_tensor)